# Why tensorflow?

1. flexibility

    1. On creating a graph, you are not bound to run the whole graph and can control the parts of the graph that are executed separately. 
    
    2. This provides a huge flexibility with your models.

2. Lazy computing
    1. representing computation without actually performing it until asked
    
3. key-feature: computational-graph approach

4. 2 important parts of this approach
    
    1. Part 1: building the GRAPH, it represents the data flow of the computations

    2. Part 2: running a SESSION, it executes the operations in the graph
    
5. One of the biggest advantages of TensorFlow is its visualizations of the computation graph, i.e. Tensorboard.

6. TensorFlow separates the definition of computations(defining ops/variables) from their execution(`session.run`). 

7. Tensorflow : flow of tensors throughout the graph, tensors: multi-dimensional array(0-D tensor: scalar, 1-D tensor: vector, 2-D tensor: matrix, ...)

In [1]:
import tensorflow as tf
import numpy as np
tf.__version__

tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# Graphs

1. all the numerical computations are expressed as a computational graph.

2. from the TensorFlow website, *"A computational graph (or graph in short) is a series of TensorFlow operations arranged into a graph of nodes"*

3. lets look at a simple graph

    1. compute $f(x, y) = x^2y + y + 2$
    
    2. <img src="simpleGraph.png" width=350/> 
    
    3. Each node in the graph is called op (short for operation). 
    
    4. We have one node for each operation; either for operations on tensors (like math operations) or generating tensors (like variables and constants, represented by the rectangular-shaped nodes, **even those are nodes**). 
    
    5. Each node takes zero or more tensors as inputs and produces a tensor as an output.

In [2]:
tf.compat.v1.reset_default_graph()

a, b = 2, 3
c = tf.add(a, b, name="ADD")

# print(dir(c))
print(c.graph)
print(c)

fw = tf.compat.v1.summary.FileWriter("./logSimple", graph=c.graph)

Tensor("ADD:0", shape=(), dtype=int32)


In [3]:
%reload_ext tensorboard
%tensorboard --logdir "./logSimple"

Reusing TensorBoard on port 6006 (pid 3705), started 3:07:48 ago. (Use '!kill 3705' to kill it.)

right now, the graph is only created, but the ops aren't executed. \
We have to do manually.

* the written code only generates the graph which only determines the expected sizes of Tensors and operations to be executed on them. 

* However, it doesn't assign a numeric value to any of the Tensors i.e. TensorFlow does not execute the graph unless it is specified to do so with a session. 

* Hence, to assign these values and make them flow through the graph, we need to create and run a session.

Therefore a TensorFlow Graph is something like a function definition in Python. It <font color="red">WILL NOT</font> do any computation for you (just like a function definition will not have any execution result). It <font color="red">ONLY</font> defines computation operations.

## Where did the names x and y come from?

* on observing the above graph, we can see that the input nodes are labelled with x and y, but they were never defined with those names.

* let's make up two names; say "Python-name" and "TensorFlow-name". 

* In this piece of code, we generated 3 variables with "Python-name"s of a, b, and c. 

* Here, a and b are Python variables, thus have no "TensorFlow-name"; while c is a Tensor with *Add* as its "TensorFlow-name".

* Tensorflow had to resort to using default names, since we didn't name them in our code.

# Session

1. session places the graph ops on hardware such as CPUs or GPUs and provides methods to execute them.

In [4]:
with tf.compat.v1.Session() as sess:
    print(sess.run(c)) # whatever nodes(ops/variables initialization) are necessary for computing c will only be run

# equivalent code
sess = tf.compat.v1.Session()
print(sess.run(c)) # whatever nodes(ops/variables initialization) are necessary for computing c will only be run
sess.close()

5
5


<font size="4" color="red">I HAVE TO USE `tf.compat.v1` BECAUSE OF THE FACT THAT THE GRAPH DEFINITIONS HAVE CHANGED IN TF-V2.0.X . PLACEHOLDERS, VARIABLES AND SESSION WERE AVAILABLE FOR TF-V1.X.X BUT NOW THEY AREN'T</font>

<font size="4">Graph with multiple math operations</font>

1. to fetch the value from pow_op, the nodes regarding add_op and mul_op have to be run, so that their output tensors van be evaluated

2. fetching the value of two tensors (i.e. output tensors of pow_op and useless_op) at the same time. 

    1. This will run the whole graph to get the required output tensors.

In [5]:
tf.compat.v1.reset_default_graph()
x = 2
y = 3
add_op = tf.add(x, y, name='Add') # x+y
mul_op = tf.multiply(x, y, name='Multiply') # x*y
pow_op = tf.pow(add_op, mul_op, name='Power') # (x+y)^(xy)
useless_op = tf.multiply(x, add_op, name='Useless') # x + x+y

with tf.compat.v1.Session() as sess:
    pow_out, useless_out = sess.run([pow_op, useless_op])
    fw = tf.compat.v1.summary.FileWriter("./logMultiple", graph=sess.graph)

In [6]:
%reload_ext tensorboard
%tensorboard --logdir "./logMultiple"

Reusing TensorBoard on port 6007 (pid 4660), started 2:38:56 ago. (Use '!kill 4660' to kill it.)

# Tensor Types

3 common ones:
       
    1. Variable    
    2. Placeholder    
    3. Constant

## Constant

1. simply used as a constant

2. values remain unchanged

3. `tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)`

In [7]:
tf.compat.v1.reset_default_graph()

# create graph
a = tf.constant(2)
b = tf.constant(3)
c = a + b
# launch the graph in a session
with tf.compat.v1.Session() as sess:
    print(sess.run(c))
    fw = tf.compat.v1.summary.FileWriter("./tfConstant", graph=sess.graph)    

5


In [8]:
%reload_ext tensorboard
%tensorboard --logdir "./tfConstant"

Reusing TensorBoard on port 6008 (pid 6325), started 2:14:50 ago. (Use '!kill 6325' to kill it.)

In [9]:
tf.compat.v1.reset_default_graph()

s = tf.constant(2.3, name='scalar', dtype=tf.float32)
m = tf.constant([[1, 2], [3, 4]], name='matrix')
# launch the graph in a session
with tf.compat.v1.Session() as sess:
    print(sess.run(s))
    print(sess.run(m))
    fw = tf.compat.v1.summary.FileWriter("./tfConstant-2", graph=sess.graph)    

2.3
[[1 2]
 [3 4]]


In [10]:
%reload_ext tensorboard
%tensorboard --logdir "./tfConstant-2"

Reusing TensorBoard on port 6009 (pid 8511), started 0:48:33 ago. (Use '!kill 8511' to kill it.)

# Variable

1. Variables are stateful nodes which output their current value; meaning that they can retain their value over multiple executions of a graph.

2. They can be saved to your disk during and after training.
    1. hence can be shared/saved among group(s) of users
    
3. By default, gradient updates (used in all neural networks) will apply to all variables in your graph. 
    1. In fact, variables are the things that you want to tune in order to minimize the loss.
    
## Better than constants
Constants are stored in the graph definition which makes them memory-expensive. In other words, constants with millions of entries makes the graph slower and resource intensive. Creation of Variables on the other hand, is an  operation. We execute these operations in the session and get the output value of the operations.

In [11]:
s = tf.compat.v1.Variable(2, name="scalar") 
m = tf.compat.v1.Variable([[1, 2], [3, 4]], name="matrix") 
W = tf.compat.v1.Variable(tf.zeros([784,10])) # 784*10 matrix

print(s, "\n\n", m, "\n\n", W)

<tf.Variable 'scalar_1:0' shape=() dtype=int32_ref> 

 <tf.Variable 'matrix_1:0' shape=(2, 2) dtype=int32_ref> 

 <tf.Variable 'Variable:0' shape=(784, 10) dtype=float32_ref>


`tf.get_variable` is a wrapper, that does the similar thing as tf.Variable

`tf.get_variable(name,
                shape=None,
                dtype=None,
                initializer=None,
                regularizer=None,
                trainable=True,
                collections=None,
                caching_device=None,
                partitioner=None,
                validate_shape=True,
                use_resource=None,
                custom_getter=None,
                constraint=None)`

In [12]:
s = tf.compat.v1.get_variable("scalar", initializer=tf.constant(2)) 
m = tf.compat.v1.get_variable("matrix", initializer=tf.constant([[0, 1], [2, 3]]))
W = tf.compat.v1.get_variable("weight_matrix", shape=(784, 10), initializer=tf.zeros_initializer())



To initialize variables, we have to invoke a variable initializer operation and run the operation on the session. 

In [13]:
tf.compat.v1.reset_default_graph()

# this is bound to given an error
a = tf.compat.v1.get_variable(name="var_1", initializer=tf.constant(2))
b = tf.compat.v1.get_variable(name="var_2", initializer=tf.constant(3))
c = tf.compat.v1.add(a, b, name="Add1")

# launch the graph in a session
with tf.compat.v1.Session() as sess:
    # now let's evaluate their value
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(c))

FailedPreconditionError: Attempting to use uninitialized value var_1
	 [[{{node _retval_var_1_0_0}}]]

1. This is because we tried to evaluate the variables before initializing them.

2. we have to run an initializer method using our session object, so as to globally initialize all variables

In [14]:
tf.compat.v1.reset_default_graph()

# this is bound to given an error
a = tf.compat.v1.get_variable(name="var_1", initializer=tf.constant(2))
b = tf.compat.v1.get_variable(name="var_2", initializer=tf.constant(3))
c = tf.compat.v1.add(a, b, name="Add1")

init = tf.compat.v1.global_variables_initializer()

# launch the graph in a session
with tf.compat.v1.Session() as sess:
    # first, initialisation
    sess.run(init)
    # now let's evaluate their value
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(c))
    fw = tf.compat.v1.summary.FileWriter("./tfVariables-1", sess.graph)

2
3
5


In [15]:
%reload_ext tensorboard
%tensorboard --logdir="./tfVariables-1"

Reusing TensorBoard on port 6010 (pid 8968), started 0:32:12 ago. (Use '!kill 8968' to kill it.)

refering to the legend provided by the graph, observe that the both `var_1` and `var_2` are highlighted as being `Namespace`, i.e. tensorflow-variables. \

Note: Variables are usually used for weights and biases in neural networks.

* Weights are usually initialized from a normal distribution using `tf.truncated_normal_initializer()`.
    
* Biases are usually initialized from zeros using `tf.zeros_initializer()`.

In [16]:
tf.compat.v1.reset_default_graph()

w = tf.compat.v1.get_variable(
    name="weight",
    shape=[2, 3],
    initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.01)
)
b = tf.compat.v1.get_variable(
    name="bias",
    shape=[3],
    initializer=tf.zeros_initializer()
)

init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session() as sess:
    sess.run(init)
    W, B = sess.run([w, b])
    print("Weights =", W)
    print()
    print("Biases =", B)

Weights = [[ 0.00758511 -0.00256374 -0.00489398]
 [ 0.0027111   0.00287458 -0.00278633]]

Biases = [0. 0. 0.]


# Placeholder

1. simply a variable that we asign data in a future time.

2. nodes whose value is fed in at execution time.

3.  we have inputs to our network that depend on some external data and we don't want our graph to depend on any real value while developing the graph, placeholders are the datatype we need.

    1. In fact, we can build the graph without any data. 
    
    2. Therefore, placeholders don't need any initial value; only a datatype (such as float32) and a tensor shape so the graph still knows what to compute with even though it doesn't have any stored values yet.

In [17]:
tf.compat.v1.reset_default_graph()
a = tf.compat.v1.placeholder(tf.float32, shape=[5])
b = tf.compat.v1.placeholder(dtype=tf.float32, shape=None, name=None)
X = tf.compat.v1.placeholder(tf.float32, shape=[None, 784], name='input')
print(a, b, X)

Tensor("Placeholder:0", shape=(5,), dtype=float32) Tensor("Placeholder_1:0", dtype=float32) Tensor("input:0", shape=(?, 784), dtype=float32)


In [18]:
tf.compat.v1.reset_default_graph()

a = tf.compat.v1.constant([5, 5, 5], tf.float32, name='A')
b = tf.compat.v1.placeholder(tf.float32, shape=[3], name='B')
c = tf.compat.v1.add(a, b, name="Add")

with tf.compat.v1.Session() as sess:
    '''
    error is expected, since although 
    we have declared a placeholder, 
    haven't assigned it a value yet, 
    hence its EMPTY
    '''
    print(sess.run(c))

InvalidArgumentError: You must feed a value for placeholder tensor 'B' with dtype float and shape [3]
	 [[node B (defined at <ipython-input-18-45c918bb6193>:4) ]]

Original stack trace for 'B':
  File "/home/laferrari/anaconda3/envs/tensorflow_cpu/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/laferrari/anaconda3/envs/tensorflow_cpu/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/laferrari/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/laferrari/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/laferrari/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 612, in start
    self.io_loop.start()
  File "/home/laferrari/anaconda3/envs/tensorflow_cpu/lib/python3.6/si

In [19]:
with tf.compat.v1.Session() as sess:
    d = {b: [1, 2, 3]} # feed dict
    print(sess.run(c, feed_dict=d))

[6. 7. 8.]


# Create a dummy neural network

In [20]:
tf.compat.v1.reset_default_graph()

# input placeholder
X = tf.compat.v1.placeholder(tf.float32, shape=[None, 784], name="input")

In [21]:
# initialize weights and bias

# create weight matrix initialized randomely from N(0, 0.01)
weight_initer = tf.compat.v1.truncated_normal_initializer(mean=0.0, stddev=0.01)
W = tf.compat.v1.get_variable(name="Weight", dtype=tf.float32, shape=[784, 200], initializer=weight_initer)

# create bias vector of size 200, all initialized as zero
bias_initer =tf.compat.v1.constant(0., shape=[200], dtype=tf.float32)
b = tf.compat.v1.get_variable(name="Bias", dtype=tf.float32, initializer=bias_initer)

In [22]:
# define computation ops

x_w = tf.matmul(X, W, name="MatMul") # create MatMul node
x_w_b = tf.add(x_w, b, name="Add")   # create Add node
h = tf.nn.relu(x_w_b, name="ReLU")   # create ReLU node

In [23]:
# Add an Op to initialize variables
init_op = tf.compat.v1.global_variables_initializer()

# launch the graph in a session
with tf.compat.v1.Session() as sess:
    sess.run(init_op) # initialize variables
    d = {X: np.random.rand(100, 784)} # create the dictionary
    print(sess.run(h, feed_dict=d)) # feed it to placeholder a via the dict 
    fw = tf.compat.v1.summary.FileWriter("./simpleNN", sess.graph)

[[-0.         -0.          0.08185606 ...  0.00225888 -0.
   0.0822422 ]
 [-0.          0.04772456  0.11408196 ...  0.15106735 -0.
  -0.        ]
 [-0.          0.0872113   0.25038424 ... -0.         -0.
  -0.        ]
 ...
 [ 0.00622779  0.02182791  0.14613493 ...  0.05204231 -0.
   0.08598535]
 [-0.         -0.          0.28011447 ...  0.00596382 -0.
  -0.        ]
 [-0.         -0.          0.22502492 ...  0.03813605 -0.
   0.00718566]]


In [25]:
%reload_ext tensorboard
%tensorboard --logdir="./simpleNN"

Reusing TensorBoard on port 6011 (pid 10018), started 0:04:24 ago. (Use '!kill 10018' to kill it.)